# Load Data and preprocess

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
#pip install ucimlrepo

In [3]:
from src.data.load_data import load_input_data
from src.data.load_data import preprocess_data
#from src.data.load_data import preprocess_data_no_onehot

In [4]:
X_train, X_test, y_train, y_test, metadata, variables, datasetdf = load_input_data()

In [5]:
import pandas as pd
from src.data.load_data import preprocess_num

num_cols = ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11',
            'Attribute13', 'Attribute16', 'Attribute18']

X_train_raw,X_test_raw = X_train,X_test
#X_train, X_test, datasetdf = preprocess_data(X_train, X_test, datasetdf, num_cols)
X_train, X_test, datasetdf = preprocess_data(X_train, X_test, datasetdf, num_cols)
X_train_num, X_test_num, datasetdf_num = preprocess_num(X_train, X_test, datasetdf, num_cols)
X_train_num_pos = X_train_num[y_train == 1]
#pd.set_option('display.max_columns', None)
#print(X_train.head())
print(datasetdf.head())

   Attribute2  Attribute5  Attribute8  Attribute11  Attribute13  Attribute16  \
0   -1.235427   -0.760138    0.945404     1.076342     2.880568     1.074000   
1    2.277975    1.028842   -0.835976    -0.747967    -1.230428    -0.724565   
2   -0.733512   -0.413341   -0.835976     0.164188     1.236170    -0.724565   
3    1.776060    1.751243   -0.835976     1.076342     0.870748    -0.724565   
4    0.270317    0.624432    0.054714     1.076342     1.601592     1.074000   

   Attribute18  class  Attribute1_A12  Attribute1_A13  ...  Attribute12_A124  \
0    -0.434114      1           False           False  ...             False   
1    -0.434114      2            True           False  ...             False   
2     2.303542      1           False           False  ...             False   
3     2.303542      1           False           False  ...             False   
4     2.303542      2           False           False  ...              True   

   Attribute14_A142  Attribute14_A143 

# Train model

In [6]:
from src.model.train_model import train_classifier
from src.model.train_model import train_classifier_LR

In [7]:
#clf = train_classifier(X_train,y_train,n_estimators=100, random_state=42) # this is for random forrest
clf = train_classifier_LR(X_train,y_train, random_state=42)

# Test model

In [8]:
from src.model.test_model import test_classifier_prediction

In [9]:
y_pred,classification_report,accuracy_score = test_classifier_prediction(clf,X_test,y_test)

In [10]:
print(accuracy_score)

0.78


In [11]:
print(classification_report)

              precision    recall  f1-score   support

           1       0.81      0.89      0.85       210
           2       0.67      0.52      0.59        90

    accuracy                           0.78       300
   macro avg       0.74      0.71      0.72       300
weighted avg       0.77      0.78      0.77       300


### User constraints / Ontology constraints

In [12]:
from src.ontology.generate_rules import *

In [13]:
X_train_positive = X_train_raw[y_train==1]
X_train_negative = X_train_raw[y_train==2]

In [14]:
positive_profile = summarize_features(X_train_positive)

In [15]:
positive_profile

{'Attribute1': {'A14': 0.49795918367346936,
  'A12': 0.23265306122448978,
  'A11': 0.20408163265306123,
  'A13': 0.0653061224489796},
 'Attribute2': {'mean': np.float64(19.240816326530613),
  'min': np.int64(4),
  'max': np.int64(60)},
 'Attribute3': {'A32': 0.5040816326530613,
  'A34': 0.3448979591836735,
  'A33': 0.09183673469387756,
  'A31': 0.03469387755102041,
  'A30': 0.024489795918367346},
 'Attribute4': {'A43': 0.2938775510204082,
  'A40': 0.23061224489795917,
  'A42': 0.17346938775510204,
  'A41': 0.12448979591836734,
  'A49': 0.09183673469387756,
  'A46': 0.03469387755102041,
  'A45': 0.02040816326530612,
  'A44': 0.014285714285714285,
  'A410': 0.00816326530612245,
  'A48': 0.00816326530612245},
 'Attribute5': {'mean': np.float64(2983.004081632653),
  'min': np.int64(338),
  'max': np.int64(15653)},
 'Attribute6': {'A61': 0.5387755102040817,
  'A65': 0.21428571428571427,
  'A62': 0.11224489795918367,
  'A63': 0.06938775510204082,
  'A64': 0.0653061224489796},
 'Attribute7': 

In [16]:
negative_profile = summarize_features(X_train_negative)

In [17]:
negative_profile

{'Attribute1': {'A11': 0.4380952380952381,
  'A12': 0.3523809523809524,
  'A14': 0.1523809523809524,
  'A13': 0.05714285714285714},
 'Attribute2': {'mean': np.float64(24.333333333333332),
  'min': np.int64(6),
  'max': np.int64(60)},
 'Attribute3': {'A32': 0.5476190476190477,
  'A34': 0.17142857142857143,
  'A33': 0.10476190476190476,
  'A31': 0.1,
  'A30': 0.0761904761904762},
 'Attribute4': {'A40': 0.2761904761904762,
  'A43': 0.2,
  'A42': 0.19523809523809524,
  'A49': 0.11428571428571428,
  'A46': 0.08571428571428572,
  'A41': 0.07142857142857142,
  'A45': 0.02857142857142857,
  'A44': 0.014285714285714285,
  'A410': 0.009523809523809525,
  'A48': 0.004761904761904762},
 'Attribute5': {'mean': np.float64(3709.233333333333),
  'min': np.int64(433),
  'max': np.int64(15945)},
 'Attribute6': {'A61': 0.7285714285714285,
  'A62': 0.12380952380952381,
  'A65': 0.09047619047619047,
  'A63': 0.03333333333333333,
  'A64': 0.023809523809523808},
 'Attribute7': {'A73': 0.3333333333333333,
  '

In [19]:
#p_statistical = extract_statistical_rules(X_train_positive, X_train_raw, num_quantiles=(0.1, 0.9), cat_threshold=0.85)
p_statistical = extract_statistical_rules(X_train_num_pos, X_train_num, num_quantiles=(0.1, 0.9), cat_threshold=0.85)

In [19]:
p_statistical

{'Attribute2': (np.float64(6.0), np.float64(36.0)),
 'Attribute5': (np.float64(929.9), np.float64(6473.9)),
 'Attribute8': (np.float64(1.0), np.float64(4.0)),
 'Attribute10': 'A101',
 'Attribute11': (np.float64(1.0), np.float64(4.0)),
 'Attribute13': (np.float64(24.0), np.float64(51.0)),
 'Attribute16': (np.float64(1.0), np.float64(2.0)),
 'Attribute18': (np.float64(1.0), np.float64(2.0))}

In [20]:
p_permutation_imp_top_features = top_features(clf, X_train, y_train, n_repeats=5, random_state=42)

In [21]:
p_permutation_imp_top_features

['Attribute1_A14',
 'Attribute9_A93',
 'Attribute6_A65',
 'Attribute2',
 'Attribute12_A124',
 'Attribute3_A34',
 'Attribute8',
 'Attribute15_A153',
 'Attribute14_A143',
 'Attribute7_A74',
 'Attribute15_A152',
 'Attribute1_A12',
 'Attribute5']

In [22]:
p = combined_features(p_permutation_imp_top_features,p_statistical)

In [23]:
p

{'Attribute2': (np.float64(6.0), np.float64(36.0)),
 'Attribute5': (np.float64(929.9), np.float64(6473.9)),
 'Attribute8': (np.float64(1.0), np.float64(4.0)),
 'Attribute10': 'A101',
 'Attribute11': (np.float64(1.0), np.float64(4.0)),
 'Attribute13': (np.float64(24.0), np.float64(51.0)),
 'Attribute16': (np.float64(1.0), np.float64(2.0)),
 'Attribute18': (np.float64(1.0), np.float64(2.0)),
 'Attribute1': 'A14',
 'Attribute9': 'A93',
 'Attribute6': 'A65',
 'Attribute12': 'A124',
 'Attribute3': 'A34',
 'Attribute15': 'A153',
 'Attribute14': 'A143',
 'Attribute7': 'A74'}

In [24]:
p_num = {key: value for key, value in p.items() if isinstance(value, tuple)}
p_cat = {key: value for key, value in p.items() if isinstance(value, str)}
print(p_num)
print(p_cat)

{'Attribute2': (np.float64(6.0), np.float64(36.0)), 'Attribute5': (np.float64(929.9), np.float64(6473.9)), 'Attribute8': (np.float64(1.0), np.float64(4.0)), 'Attribute11': (np.float64(1.0), np.float64(4.0)), 'Attribute13': (np.float64(24.0), np.float64(51.0)), 'Attribute16': (np.float64(1.0), np.float64(2.0)), 'Attribute18': (np.float64(1.0), np.float64(2.0))}
{'Attribute10': 'A101', 'Attribute1': 'A14', 'Attribute9': 'A93', 'Attribute6': 'A65', 'Attribute12': 'A124', 'Attribute3': 'A34', 'Attribute15': 'A153', 'Attribute14': 'A143', 'Attribute7': 'A74'}


### Final outputs of Input section of pipeline
For experimentation, here are some values

In [25]:
from src.utilities.desired_space import desired_space
x= X_test[0:1] #test instance #select first row as dataFrame
p_num = p_num
p_cat = p_cat
#p= {
    #'Attribute5':(0, 1) #credit amount #I put a normalized amount here
#}#perturbation map
#desired_space= desired_space(2, datasetdf) #all the feature vectors with output=2 #not preprocessed! Potential issue from authors!
#print(datasetdf.head())
desired_space = desired_space(1,datasetdf) #did preprocessing also to datasetdf
#print(X_train.head())
#print(desired_space.head())
cat_f= X_train.select_dtypes(include=['bool']) #categorical features
#print(cat_f)
num_f= X_train.select_dtypes(include=['int64', 'float64']) #numerical features
#print(num_f)
protect_f= ['Attribute9_A92'  'Attribute9_A93'  'Attribute9_A94', 'Attribute5'] # Personal status and sex (X_train.head() shows only 3 values (columns) while there are 5) #protected features
features= X_train.columns #all features
t= 2 #desired outcome
f= clf #black-box model for which we want to find CF
X= X_train #training data
step = {attribute:0.01 for attribute in p_num.keys()}
#step= {
#    'Attribute5': 0.01 #credit amount steps #tiny amount bc of normalized feature values
#} #dictionary holding the feature distribution to be used for single feature perturbation method

### Derived variables

In [26]:
f2change = p.keys()

# UFCE

### Feature Selection: Mutual Information (MI)

In [27]:
from itertools import combinations
from src.utilities.mutual_information import mi_score
from src.utilities.mutual_information import MI
feature_pairs = list(combinations(list(features), 2))
print(feature_pairs[:5])
mi_list = MI(feature_pairs, X_train) #used all features available #original authors do over entire X but it is not preprocessed... Which i think is nessecary? #also works for bool one-hot encoding 
# top 5 
print(mi_list[:5])

[('Attribute2', 'Attribute5'), ('Attribute2', 'Attribute8'), ('Attribute2', 'Attribute11'), ('Attribute2', 'Attribute13'), ('Attribute2', 'Attribute16')]
[('Attribute9_A92', 'Attribute9_A93'), ('Attribute2', 'Attribute5'), ('Attribute3_A32', 'Attribute3_A34'), ('Attribute17_A172', 'Attribute17_A173'), ('Attribute12_A124', 'Attribute15_A153')]


### Nearest Neighbour: KD Tree

In [28]:
from src.utilities.nearest_neighbours import FNN
nn = FNN(desired_space,x,3.5) #radius arbitrarily set; can experiment with how close we want it to be to x #3.5 for euclidean distance is around 0.5 for each of the 48 features difference from x, get 3 datapoints
print(nn)

     Attribute2  Attribute5  Attribute8  Attribute11  Attribute13  \
652    0.270317   -0.335901    0.945404    -1.660121     0.870748   
622    2.277975    0.266786    0.945404    -0.747967     0.231260   
301    1.274146    0.225634    0.945404    -1.660121     0.596682   

     Attribute16  Attribute18  Attribute1_A12  Attribute1_A13  Attribute1_A14  \
652    -0.724565    -0.434114           False           False           False   
622    -0.724565    -0.434114           False           False            True   
301    -0.724565    -0.434114            True           False           False   

     ...  Attribute12_A124  Attribute14_A142  Attribute14_A143  \
652  ...             False             False              True   
622  ...             False             False             False   
301  ...             False             False              True   

     Attribute15_A152  Attribute15_A153  Attribute17_A172  Attribute17_A173  \
652              True             False             Fa

### Subspace: Intervals

In [29]:
from src.utilities.nearest_neighbours import intervals
from src.utilities.nearest_neighbours import intervals2
#subspace = intervals(nn,p,f2change,x) 
subspace = intervals2(nn,p_num,p_cat,f2change,x)
print(subspace) #correct output

6.0
36.0
929.9
6473.9
1.0
4.0
1.0
4.0
24.0
51.0
1.0
2.0
1.0
2.0
subspace cat val.:  A101
subspace cat val.:  A14
subspace cat val.:  A93
subspace cat val.:  A65
subspace cat val.:  A124
subspace cat val.:  A34
subspace cat val.:  A153
subspace cat val.:  A143
subspace cat val.:  A74
{'Attribute2': [np.float64(6.0), np.float64(2.2779745367108846)], 'Attribute5': [np.float64(929.9), np.float64(0.26678599187094654)], 'Attribute8': [np.float64(1.0), np.float64(0.9454035729938243)], 'Attribute11': [np.float64(1.0), np.float64(-0.7479667044950183)], 'Attribute13': [np.float64(24.0), np.float64(0.8707481112693376)], 'Attribute16': [np.float64(1.0), np.float64(-0.7245647372489964)], 'Attribute18': [np.float64(1.0), np.float64(-0.4341140500939406)], 'Attribute10': 'A101', 'Attribute1': 'A14', 'Attribute9': 'A93', 'Attribute6': 'A65', 'Attribute12': 'A124', 'Attribute3': 'A34', 'Attribute15': 'A153', 'Attribute14': 'A143', 'Attribute7': 'A74'}


### CF Generator: Single Feature

In [30]:
from src.utilities.UFCE import SF
#cf = SF(x,X,cat_f,p,f,t,step)
cf = SF(x,X_train,p_num,p_cat,f,t,step) # changed from X_train_raw to X_train
print(cf)

original x prediction:  [1]
all columns are 0 for this category, as it is the reference feature value


C:\Users\Jurjen\Desktop\Python exercises code (target level)\cf-enhancement-xai\src\utilities\UFCE.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  z.loc[:, relevant_columns] = 0
C:\Users\Jurjen\Desktop\Python exercises code (target level)\cf-enhancement-xai\src\utilities\UFCE.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  z.loc[:, relevant_columns] = 0
C:\Users\Jurjen\Desktop\Python exercises code (target level)\cf-enhancement-xai\src\utilities\UFCE.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype fi

[]


### CF Generator: Double Features

In [31]:
from src.utilities.UFCE import DF
z = DF(datasetdf,x,X_train,subspace,mi_list,cat_f,num_f,features,protect_f,f,t)
print(z)

going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
[[ 2.88056839]
 [-1.23042764]
 [ 1.23616998]
 [ 0.87074811]
 [ 1.60159185]
 [-0.04280656]
 [ 1.60159185]
 [-0.04280656]
 [ 2.33243559]
 [-0.68229483]
 [-0.95636123]
 [-1.0477167 ]
 [-1.23042764]
 [ 2.24108012]
 [-0.68229483]
 [-0.31687296]
 [ 1.60159185]
 [-0.95636123]
 [ 0.77939264]
 [-0.40822843]
 [ 1.14481451]
 [ 0.77939264]
 [ 1.14481451]
 [ 0.77939264]
 [-0.86500577]
 [ 0.04854891]
 [ 0.32261531]
 [ 0.59668171]
 [-0.13416203]
 [ 2.51514652]
 [ 0.04854891]
 [-0.7736503 ]
 [-0.4995839 ]
 [ 1.96701372]
 [-0.2255175 ]
 [-0.95636123]
 [-0.40822843]
 [ 0.13990437]
 [ 0.13990437]
 [-1.0477167 ]
 [-0.4995839 ]
 [-0.86500577]
 [ 0.77939264]
 [-1.0477167 ]
 [ 2.05836919]
 [-0.04280656]
 [ 0.32261531]
 [-1.13907217]
 [ 0.32261

C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation

going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
g

C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation

going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
g

C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation

going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
going to next feature pair
[[ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [ 2.87256516]
 [-0.72456474]
 [-0.72456474]
 [ 2.87256516]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [ 1.07400021]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 [ 1.07400021]
 [-0.72456474]
 [-0.72456474]
 

C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Users\Jurjen\Desktop\Python exercises code (target level)\venv\lib\site-packages\sklearn\utils\validation

In [32]:
print("original: ")
print(x)
print("counterfactual: ")
print(z)

original: 
     Attribute2  Attribute5  Attribute8  Attribute11  Attribute13  \
115    2.277975    0.141086    0.945404    -1.660121     1.053459   

     Attribute16  Attribute18  Attribute1_A12  Attribute1_A13  Attribute1_A14  \
115    -0.724565    -0.434114           False           False            True   

     ...  Attribute12_A124  Attribute14_A142  Attribute14_A143  \
115  ...             False             False              True   

     Attribute15_A152  Attribute15_A153  Attribute17_A172  Attribute17_A173  \
115              True             False             False              True   

     Attribute17_A174  Attribute19_A192  Attribute20_A202  
115             False              True             False  

[1 rows x 48 columns]
counterfactual: 
     Attribute2  Attribute5  Attribute8  Attribute11  Attribute13  \
115    2.277975    0.141086    0.945404    -1.660121     1.053459   

     Attribute16  Attribute18  Attribute1_A12  Attribute1_A13  Attribute1_A14  \
115    -0.72456

In [34]:
print("Differences between original and counterfactual:")

# Compare x and z for all columns
differences = (x != z).iloc[0]

# Print only the differing columns with their values in x and z
for col in x.columns[differences]:
    print(f"{col}: original = {x.iloc[0][col]}, counterfactual = {z.iloc[0][col]}")

Differences between original and counterfactual:


# Evaluation

### Metrics: Plausible

### Metrics: Actionable

### Metrics: Feasible

### Comparison against author's dataset

In [35]:
readpath = r'C:\Users\Jurjen\Desktop\Python exercises code (target level)\cf-enhancement-xai\src\data\bank.csv'
df = pd.read_csv(readpath)
print(df.head())  # to display the first few rows
print(len(df.columns))

   Unnamed: 0  age  Experience  Income  Family     CCAvg  Education  Mortgage  \
0           0   52          15     140       2  7.737671          2        32   
1           1   51           0     150       3  5.073039          1       328   
2           2   32          18     144       3  6.295617          3        59   
3           3   51          20     159       1  5.495390          3       275   
4           4   38          17     154       2  3.225856          3       189   

   SecuritiesAccount  CDAccount  Online  CreditCard  Personal Loan  
0                  1          1       1           0              1  
1                  1          1       0           1              1  
2                  0          1       0           1              1  
3                  0          1       1           0              1  
4                  1          1       1           1              1  
13


In [ ]:
from src.data.load_data import preprocess_bank
features, catf, numf, uf, f2change, outcome_label, desired_outcome, nbr_features, protectf, data_lab0, data_lab1 = preprocess_bank(df)